In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/dictionary_sentipolc_8266_training.joblib")

In [3]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [4]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["opos"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
[0]


In [5]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate, Dropout
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

In [6]:
import joblib
import pandas as pd
emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_2000_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/dictionary_sentipolc_8266_test.joblib")

In [7]:
# define example
data = dataset_test["opos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 1 0 ... 1 0 1]
[0 1 0 ... 1 0 1]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [13]:
import keras
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np

input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(8266,))

#Bi-LSTM
#bi = Bidirectional(LSTM(64, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True) (bi)

app = Add() ([bi, att])                      
flat = Flatten()(app)
den1 = Dense(4000) (flat)
den2 = Dense(2000) (den1)
den3 = Dense(1000) (den2)
den4 = Dense(500) (den3)
den5 = Dense(256) (den4)
den6 = Dense(128) (den5)
den7 = Dense(64) (den6)

#WMAL 
d0 = Dense(8266) (input_wmal)

#Concat
concat = Concatenate(axis=1)([den7, d0])
final_dense = Dense(32, activation="relu") (concat)
output = Dense(2,activation="softmax") (final_dense)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.RMSprop()

m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

class myCallback(keras.callbacks.Callback):
    bestF1=0
    def __init__(self):
        super().__init__()
        self._get_pred = None
        self.preds = []
    def on_epoch_end(self,epoch, logs=None):
        predictions = m.predict([emb_matrix_test,wmal_matrix_test])
        predictions_vec = np.zeros((2000,2))
        i = 0
        confidences= []
        for item in predictions:
          index = np.argmax(item);
          confidences.append(item[index])
          predictions_vec[i][index] = 1
          i = i+1
        labels_to_eval = []
        for vec in predictions_vec:
          inverted = label_encoder.inverse_transform([argmax(vec)])[0]
          labels_to_eval.append(inverted)
        
        precision,recall,fscore,support=score(dataset_test["opos"],labels_to_eval,average='macro')
        print("Test f1-score:"+str(fscore))
        if fscore > self.bestF1:
          m.save("bestModel.h5")
          print("BEST MODEL!!!!!")
          self.bestF1 = fscore

callbacks = [myCallback()]

In [ ]:
m.summary()

In [14]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=100,validation_split=0.15, callbacks=[callbacks])
#m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_neg_sentipolc_0.72803.h5")

50/50 [==============================] - ETA: 0s - loss: 1.8558e-06 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [15]:
m.load_weights("bestModel.h5")
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [16]:
import numpy as np
predictions_vec = np.zeros((2000,2))
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [17]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["opos"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.88882   0.89260   0.89071      1648
           1    0.48696   0.47727   0.48207       352

    accuracy                        0.81950      2000
   macro avg    0.68789   0.68493   0.68639      2000
weighted avg    0.81809   0.81950   0.81878      2000



In [18]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_new_pos_sentipolc_0.68639.h5")

In [19]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_SENTIPOLC_pos_full_0.68639.csv");